In [2585]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.signal import savgol_filter

In [2586]:
data = pd.read_csv('Sorghum_Hyperspectral_Data_1820.csv')


C:\Users\Deniz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2587]:
most_important_index_SLA =pd.read_csv('most_important_index.csv')

In [2588]:
data.head()

,crop,PlotID,year,trt,pheno,CHL,EWT,LWC,SLA,N,...,X2491,X2492,X2493,X2494,X2495,X2496,X2497,X2498,X2499,X2500
0,sorghum,281,2018,greenhouse,yes,446.944,0.021140,0.801818,191.380472,2.84,...,0.057221,0.057268,0.057137,0.056709,0.056457,0.056115,0.055923,0.055987,0.055888,0.055737
1,sorghum,282,2018,greenhouse,yes,713.800,0.013651,0.718788,187.244828,3.44,...,0.065625,0.065636,0.065593,0.065518,0.065329,0.065109,0.064884,0.064390,0.064180,0.064182
2,sorghum,283,2018,greenhouse,yes,724.911,0.013870,0.715429,181.260442,3.52,...,0.061687,0.061462,0.061244,0.061186,0.060975,0.060398,0.060287,0.060618,0.060535,0.060500
3,sorghum,284,2018,greenhouse,yes,738.400,0.016723,0.753034,182.327753,4.13,...,0.055341,0.055124,0.054997,0.055162,0.055078,0.054760,0.054617,0.054473,0.054353,0.054351
4,sorghum,285,2018,greenhouse,yes,700.344,0.023406,0.798125,168.909288,3.85,...,0.056971,0.057206,0.057154,0.056835,0.056743,0.056700,0.056552,0.056538,0.056412,0.056039


In [2589]:
data.year.value_counts(dropna= False)

2020    1470
2018     321
Name: year, dtype: int64

In [2590]:
data.shape

(1791, 2171)

In [2591]:
data.columns[1:50]

Index(['PlotID', 'year', 'trt', 'pheno', 'CHL', 'EWT', 'LWC', 'SLA', 'N', 'P',
       'K', 'Mg', 'Ca', 'S', 'Fe', 'Mn', 'B', 'Cu', 'Zn', 'X350', 'X351',
       'X352', 'X353', 'X354', 'X355', 'X356', 'X357', 'X358', 'X359', 'X360',
       'X361', 'X362', 'X363', 'X364', 'X365', 'X366', 'X367', 'X368', 'X369',
       'X370', 'X371', 'X372', 'X373', 'X374', 'X375', 'X376', 'X377', 'X378',
       'X379'],
      dtype='object')

In [2592]:
data.loc[: , 'trt'].value_counts(dropna = False)

HN            738
LN            732
greenhouse    321
Name: trt, dtype: int64

In [2593]:
#define the trait for analysis
trait = 'Mo'


In [2594]:
#Drop NAs 
data = data.loc[data[trait].notnull(), :]
data.shape

KeyError: 'Mo'

In [ ]:
##Choose the year
##data= data.loc[data['year'] ==  2020 , :]

In [ ]:
sns.boxplot(y= trait, x='year', data=data)

In [ ]:
#defining a function to remove the outliers
def outlier_treatment(datacolumn):
 sorted(datacolumn)
 Q1=datacolumn.quantile(0.25)
 Q3=datacolumn.quantile(0.75)
 #Q1,Q3 = np.percentile(datacolumn , [25,75])
 IQR = Q3 - Q1
 lower_range = Q1 - (1.5 * IQR)
 upper_range = Q3 + (1.5 * IQR)
 return lower_range,upper_range

In [ ]:
outlier_treatment(data[trait])


In [ ]:
lowerbound,upperbound = outlier_treatment(data[trait])

In [ ]:
rm_index =data[(data[trait] < lowerbound) | (data[trait] > upperbound)].index
rm_index

In [ ]:
data.drop(rm_index, axis=0 ,inplace=True)

In [ ]:
# choose the years for grouping in the plots
year = data.loc[: , 'year']

In [ ]:
data.shape


In [ ]:
# define trait data
y = data.loc[: , trait]


In [ ]:
#select the columns of HR data
spec_columns = [col for col in data if col.startswith('X')]


In [ ]:
# Define the X by choosing only the predictors
X = data[spec_columns]


In [ ]:
# Apply Savitzky Golay Filter
X = savgol_filter(X, window_length=5 , polyorder = 2)
X = pd.DataFrame(X, columns= spec_columns)

In [ ]:
# Remove first 100 wawelengths
X = X.drop(X.columns[0:100], axis=1)

In [ ]:
# Create a np array for downsampling for every 5 nm
column_filter = np.arange(0, 2050, 5)

In [ ]:
# Choose from the X based on filtering array
X = X.iloc[: , column_filter]


In [ ]:
X.head()

In [ ]:
Index = pd.Series(most_important_index_SLA.iloc[: , 1])

In [ ]:
X_reduced = X.loc[:, Index]

In [ ]:
#define the cv
cv = RepeatedKFold(n_splits=10)
mse = []

#Calculate MSE using cross-validation, adding one component at a time
for i in np.arange(1, 40):
    pls = PLSRegression(n_components=i)
    score = -1*model_selection.cross_val_score(pls, X_reduced, y, cv=cv,
               scoring='neg_mean_squared_error').mean()
    mse.append(np.sqrt(score))

n_comp = mse.index(min(mse)) +1 

In [ ]:
# split the dataset into training (60%) and testing (40%) sets
X_train,X_test,y_train,y_test = train_test_split(X, y ,test_size=0.4) 

In [ ]:
X.shape

In [ ]:
# Train the model
pls = PLSRegression(n_components= 20) #n_comp)
pls.fit(X_train, y_train)

In [ ]:
# Calculate the RMSE
np.sqrt(mean_squared_error(y_test, pls.predict(X_test)))

In [ ]:
y_pred = pls.predict(X_test)

In [ ]:
y_pred = pd.DataFrame(y_pred)

In [ ]:
# Combine the 'year' and y_test for plotting and reset the index for combining with y_pred
y_test =pd.merge(y_test, year , left_index=True , right_index=True,  how='left').reset_index(drop=True)

In [ ]:
y_test.head()

In [ ]:
type(y_pred)

In [ ]:
#Combine y_test and y_pred and get one data frame for seaborn scatter plot
data_plot = pd.concat([y_test, y_pred], axis = 1)

In [ ]:
data_plot =data_plot.rename({0 : 'predicted'} ,axis =1)

In [ ]:
data_plot.head()

In [ ]:
# Calculate the spearman correlation , r2 and p value and pass them into the 'text'

from scipy.stats import pearsonr, spearmanr
#y_pred2 = [x[0] for x in data_plot.predicted]
r, p = spearmanr(data_plot[trait], data_plot.predicted)
r2 = r**2
text =r2, p
text = [np.round(r2, 2) for r2 in text]


In [ ]:
#plot the results
text1 = 'r2=%s, p=%s' % (text[0], text[1])
#sns.lmplot(x=trait, y='predicted',data=data_plot)
sns.regplot(x=trait, y='predicted' , scatter_kws={'s' : 15},data=data_plot)
xmin , xmax , ymin, ymax = plt.axis()
sns.scatterplot(x=trait, y='predicted', hue= 'year', data=data_plot).text((xmax)/2, ymax*0.96, text1, fontsize=12)
plt.xlabel('Ground Truth' + ' ' + trait)
plt.ylabel('Predicted' + ' ' + trait + ' ' + 'Values')
plt.title(trait + ' ' + 'PLSR')



In [ ]:
data_2022 = pd.read_csv('sorghum_2022_whole.csv')

In [ ]:
data_2022 =data_2022.loc[data_2022['355'].notnull(), :]

In [ ]:
data_2022['CHL'] = data_2022[['CHLp1', 'CHLp2', 'CHLp3']].mean(axis=1)

In [ ]:
data_2022['LWC'] = data_2022['LWC']/100

In [ ]:
data_2022 = data_2022.loc[data_2022['Leaf_num'] != 2 , :]

In [ ]:
#data_2022 = data_2022.loc[data_2022['Leaf_pos'] == 3 , :]

In [ ]:
#data_2022 = data_2022.loc[data_2022['CHL']< 700 , :]

In [ ]:
sns.boxplot(y= trait, data=data_2022)

In [ ]:
import copy
data_test = copy.deepcopy(data_2022)

data = pd.read_csv
my_group = data_test.groupby(['Genotype', 'Rep']).mean()

my_group.head()

In [ ]:
my_group = my_group.reset_index(level=['Genotype', 'Rep'])

In [ ]:
outlier_treatment(my_group[trait])

In [ ]:
lowerbound,upperbound = outlier_treatment(my_group[trait])

In [ ]:
rm_index =my_group[(my_group[trait] < lowerbound) | (my_group[trait] > upperbound)].index
rm_index

In [ ]:
my_group.drop(rm_index, axis=0 ,inplace=True)

In [ ]:
y_test = my_group.loc[:, trait]

In [ ]:
y_test.shape

In [ ]:
X_total = my_group.iloc[:, 13:2164]

In [ ]:
X_total.drop(columns = X_total.columns[0:100], inplace = True) 

In [ ]:
X_total

In [ ]:
#select the columns of HR data
spec_columns2 = X_total.columns

In [ ]:
# Apply Savitzky Golay Filter
X_total = savgol_filter(X_total, window_length=5 , polyorder = 2)
X_total = pd.DataFrame(X_total, columns= spec_columns2)

In [ ]:
X_total.head()

In [ ]:
column_filter = np.arange(0,2050,5)  ## creating an index for column filtering
X_test= X_total.iloc[:, column_filter] # selecting the filtered columns 

In [ ]:
X_test.columns = X.columns

In [ ]:
X_test= X_test.loc[:,Index]

In [ ]:
X_test.shape

In [ ]:
#define the cv
cv = RepeatedKFold(n_splits=10)
mse = []

In [ ]:
#Calculate MSE using cross-validation, adding one component at a time
#for i in np.arange(1, 40):
 #   pls = PLSRegression(n_components=i)
  #  score = -1*model_selection.cross_val_score(pls, X, y, cv=cv,
   #            scoring='neg_mean_squared_error').mean()
    #mse.append(np.sqrt(score))

In [ ]:
#n_comp = mse.index(min(mse)) +1 

In [ ]:
rsqu = [] 
for i in np.arange(1, 50):
    pls = PLSRegression(n_components= i)
    pls.fit(X_reduced, y)
    y_pred = pls.predict(X_test)
    r , p = spearmanr(y_test , y_pred)
    rsqu.append(r**2)
    n_comb = rsqu.index(max(rsqu)) +1

In [ ]:
# Train the model
pls = PLSRegression(n_components=n_comb)
pls.fit(X_reduced, y)

In [ ]:
np.sqrt(mean_squared_error(y_test, pls.predict(X_test)))

In [ ]:
y_pred = pls.predict(X_test)

In [ ]:
plt.scatter(y_test, y_pred)
plt.ylabel('Predicted' + ' ' + trait +  ' ' 'values')
plt.xlabel('Ground truth LWC measurements')
sns.regplot(x = y_test , y = y_pred)


In [ ]:
from scipy.stats import pearsonr, spearmanr
y_pred2 = [x[0] for x in y_pred]

r, p = spearmanr(y_test, y_pred2)
r2 = r**2
print(r2, p)

In [ ]:
rsqu